In [156]:
import numpy as np
import tensorflow as tf
import random
from tensorflow.python.platform import gfile
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from tensorflow.python.ops import io_ops
from matplotlib import pyplot as plt
from itertools import count
from time import sleep
import os
%matplotlib 

Using matplotlib backend: Qt5Agg


In [117]:
tf.reset_default_graph()
try:
    isess.close()
except NameError:
    pass
isess = tf.InteractiveSession()

In [118]:
FLAGS = {}
FLAGS['dataset_dir'] = r"../../Temp_datasets/commands/audio"
FLAGS['wanted_words'] = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine'] 

FLAGS['background_dir'] = "_background_noise_"
FLAGS['clip_time_ms'] = 1500
FLAGS['window_time_ms'] = 30
FLAGS['window_stride_ms'] = 10

FLAGS['sampling_rate'] = 16000
FLAGS['mel_num_bins'] = 40
FLAGS['mel_f_min'] = 40
FLAGS['mel_f_max'] = FLAGS['sampling_rate']/2

FLAGS['lr'] = 1e-4
FLAGS['batch_size'] = 128
#dataset is extended by this percent
FLAGS['unknown_percent'] = 10
FLAGS['silence_percent'] = 40

FLAGS['valid_percent'] = 10
FLAGS['train_log'] = 'train_log'
FLAGS['valid_log'] = 'valid_log'
FLAGS['checkpoint_dir'] = 'checkpoints/conv.ckpt'
FLAGS['background_volume'] = 0.8
FLAGS['target_rms'] = 0.1
FLAGS['max_shift_ms'] = 400
FLAGS['background_percent'] = 80


#override settings
flags_dev ={
#     "allow_no_examples":None,
#     'dataset_dir':  r"dev_audio",
#     'wanted_words': ['nine', 'seven'],
#     'batch_size':   2
}

for flag in flags_dev:
    FLAGS[flag] = flags_dev[flag]
    pass

In [146]:
import pyaudio 
import wave
from datetime import datetime

def play_audio(fs, data):
    from io import BytesIO
    from scipy.io import wavfile
      
    
    if data.dtype == np.float32 or data.dtype == np.float64:
        max_abs = np.max(np.abs(data))
        if max_abs > 1.0:
            print("Loudest point was {}, so data will be scaled...".format(max_abs))
            data = data/max_abs
        data = (data*(1<<15)).astype(np.int16)
    elif data.dtype == np.int16:
        pass
    else:
        raise RuntimeError("Data type unsupported")
        
    with BytesIO() as buffer:
        p = pyaudio.PyAudio()
        wavfile.write(buffer, fs, data)
        f = wave.open(buffer,"rb")  
        #define stream chunk   
        chunk = 1024  
        
        stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                        channels = f.getnchannels(),  
                        rate = f.getframerate(),  
                        output = True)  
        
        #read data  
        audio_data = f.readframes(chunk)

        #play stream  
        while audio_data:  
            stream.write(audio_data)  
            audio_data = f.readframes(chunk)  

        #stop stream  
        stream.stop_stream()  
        stream.close()  
        p.terminate()
        
def record_audio(seconds, rate, countdown=True):        
    from time import sleep
    FORMAT = pyaudio.paFloat32
    CHANNELS = 1
    CHUNK = 1024
    
    audio = pyaudio.PyAudio()
    
    if countdown:
        for i in range(3,0,-1):
            print(i)
            sleep(1)
            
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=rate, input=True,
                    frames_per_buffer=CHUNK)
    print("recording...")
    t_start = datetime.now()
    frames = []
    for i in range(0, int(rate / CHUNK * seconds)):
        data = stream.read(CHUNK)
        frames.append(data)
    t = datetime.now() - t_start
    print("finished recording, took {}".format(t.seconds + t.microseconds/1e6))

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    raw_bytes = b''.join(frames)
    samples = np.frombuffer(raw_bytes, dtype=np.float32)
    return samples

In [120]:
def get_model_settings(clip_time_ms, sampling_rate, window_time_ms, window_stride_time_ms, mel_f_min, mel_f_max, mel_num_bins, 
                       background_volume, background_percent, max_shift_ms):
    clip_samples = int(clip_time_ms * sampling_rate / 1000)
    window_samples = int(window_time_ms * sampling_rate / 1000)
    max_shift_samples = int(max_shift_ms * sampling_rate / 1000)
    window_stride_samples = int(window_stride_time_ms * sampling_rate / 1000)
    length_in_samples = 1 + int((clip_samples - window_samples) / window_stride_samples)
    fft_size = int(2**np.ceil(np.log2(window_samples)))
    settings = {
        'clip_samples':          clip_samples,
        'window_samples':        window_samples,
        'window_stride_samples': window_stride_samples,
        'fingerprint_size':      length_in_samples*mel_num_bins,
        'sampling_rate':         sampling_rate,
        'mel_f_min':             mel_f_min,
        'mel_f_max':             mel_f_max,
        'mel_num_bins':          mel_num_bins,
        'lenght_in_samples':     length_in_samples,
        'fft_size':              fft_size,
        'background_volume':     background_volume,
        'background_percent':    background_percent,
        'max_shift_samples':     max_shift_samples
        
    }
    return settings

def get_training_settings(batch_size, valid_percent, unknown_percent, silence_percent):
    settings = {
        'batch_size':          batch_size,
        'valid_percent':       valid_percent,
        'unknown_percent':     unknown_percent,
        'silence_percent':     silence_percent
    }
    return settings

model_settings = get_model_settings(FLAGS['clip_time_ms'], FLAGS['sampling_rate'], FLAGS['window_time_ms'],
                                    FLAGS['window_stride_ms'], FLAGS['mel_f_min'], FLAGS['mel_f_max'], FLAGS['mel_num_bins'],
                                    FLAGS['background_volume'], FLAGS['background_percent'], FLAGS['max_shift_ms'])
training_settings = get_training_settings(FLAGS['batch_size'], FLAGS['valid_percent'], FLAGS['unknown_percent'], 
                                          FLAGS['silence_percent'])
    

In [121]:
#TODO: clip->normalize mixed signal, random shift
class AudioProcessor:
    def __init__(self, dataset_dir, background_dir, wanted_words, model_settings, training_settings):
        self._model_settings = model_settings
        self._mel_matrix = tf.constant(self.prepare_to_mel_matrix())
        self.unknown_label, self.silence_label = '_unknown_', '_silence_'
        self.id_to_label =  wanted_words + [self.unknown_label, self.silence_label]
        self.label_to_id = {label:i for i, label in enumerate(self.id_to_label)}
        self.train_set, self.valid_set, background_paths = self.load_audio(wanted_words, dataset_dir, background_dir, training_settings)
        self._next_background = self.process_backgrounds(background_paths)
        self.x_src, self.y_src, self.iter = self.prepare_datasets(len(wanted_words), training_settings)
        self._signals, self._labels = self.iter.get_next()
        
    def prepare_to_mel_matrix(self):
        print("Preparing mel matrix")
        mel_num_bins = self._model_settings['mel_num_bins']
        mel_f_min = self._model_settings['mel_f_min']
        mel_f_max = self._model_settings['mel_f_max']
        num_spectrogram_bins = self._model_settings['fft_size']//2+1
        fs = self._model_settings['sampling_rate']

        linear_to_mel_weight_matrix = tf.contrib.signal.linear_to_mel_weight_matrix(
          mel_num_bins, num_spectrogram_bins, fs, mel_f_min, mel_f_max)
        with tf.Session() as sess:
            out = linear_to_mel_weight_matrix.eval(session=sess)
        print("Preparing mel matrix finished")
        return out
    
    def load_audio(self, wanted_words, dataset_dir, background_dir, training_settings):
        """
            Finds all wave files in dataset directory and arranges them into word -> list of files
        """
        print("Indexing audio files")
        wanted_words = {word: 0 for word in wanted_words}
        index = []
        unknown_index = []
        background_paths = []
        search_path = os.path.join(dataset_dir, '*', '*.wav')
        for wav_path in gfile.Glob(search_path):
            _, word = os.path.split(os.path.dirname(wav_path))
            if word == background_dir:
                background_paths.append(wav_path)
                continue
            if word in wanted_words:
                wanted_words[word] += 1
                index.append((wav_path, self.label_to_id[word]))
            else:
                unknown_index.append((wav_path, self.label_to_id[self.unknown_label]))
        
        if not all(wanted_words.values()) and "allow_no_examples" not in FLAGS:
            raise RuntimeError("Didn't find any audio for {}".format([w for w in wanted_words if not wanted_words[w]]))
        
        #append some unknown words
        random.shuffle(unknown_index)
        unknown_elems = min( int(round(len(index) * training_settings['unknown_percent'] / 100)), len(unknown_index) )
        index.extend(unknown_index[:unknown_elems])
        
        #append silence - volume will be set to 0, so any filename is ok
        silence_elems = int(len(index) * training_settings['silence_percent'] / 100)
        silence_element = (index[0][0], self.label_to_id[self.silence_label]) # (some .wav file, silence index)
        for _ in range(silence_elems):
            index.append(silence_element)
        
        random.shuffle(index)
        valid_size = int(len(index)*training_settings['valid_percent']/100)
        
        #list of tuples -> tuple of lists
        valid = list(zip(*index[0:valid_size]))
        train = list(zip(*index[valid_size:] ))
        
    
        print("Examples per word:")
        print( {**wanted_words, self.silence_label: silence_elems, self.unknown_label:unknown_elems} )
        print("Indexing audio files finished")
        return train, valid, background_paths
    
    def load_wav(self, path, desired_samples=-1):
        wav_loader = io_ops.read_file(path)
        audio, fs = contrib_audio.decode_wav(wav_loader, desired_channels=1, desired_samples=desired_samples)
        tf.Assert(tf.equal(fs, self._model_settings['sampling_rate']), [fs])
        audio = tf.squeeze(audio)
        return audio
    
    def process_backgrounds(self, background_paths):
        print("Loading backgrounds")
        path_input = tf.placeholder(tf.string)
        wav_out = self.load_wav(path_input)
        rms_scale_factor = (tf.sqrt(tf.reduce_mean(wav_out**2)) / FLAGS['target_rms'])
        wav_out = tf.clip_by_value(wav_out / rms_scale_factor, -1, 1)
        with tf.Session() as sess:
            wavs = [ sess.run(wav_out, feed_dict={path_input:path}) for path in background_paths ]
            
        #to limit RAM usage, instead of using dataset.from_tensor_slices (which creates huge copies of
        #of wav_out, taking more than x4 memory than they should) we create generator that returns data
        #wav_out, that is referenced in this generator, is never freed since tf doesn't delete datasets
        #this causes about 35 MB leak per AudioProcessor creation
        #UPDATE - this leak should no longer happen - TODO check this
        def gen():
            num_examples = len(wavs)
            clip_samples = self._model_settings['clip_samples']
            while True:
                example_no = random.randint(0, num_examples-1)
                wav_data = wavs[example_no]
                random_offset = random.randint(0, wav_data.size-1-clip_samples)
                yield wav_data[random_offset:random_offset+clip_samples]
                
        next_background = tf.data.Dataset.from_generator(gen, tf.float32, output_shapes=(self._model_settings['clip_samples'],))\
                          .make_one_shot_iterator().get_next()
        print("Loading backgrounds finished")
        return next_background
        
    def prepare_datasets(self, num_labels, training_settings):
        clip_samples = self._model_settings['clip_samples']
        
        x_src = tf.placeholder(tf.string, (None,))
        y_src = tf.placeholder(tf.int32,  (None,))
        x_data = tf.data.Dataset.from_tensor_slices(x_src).map(lambda path: self.load_wav(path, clip_samples))
        y_data = tf.data.Dataset.from_tensor_slices(y_src)
        dataset = tf.data.Dataset.zip((x_data, y_data)).batch(training_settings['batch_size'])
            
        return x_src, y_src, dataset.make_initializable_iterator()
    
    def signal_to_mel(self, signals, debug_stuff = False):
        """
            signals: [num_signals, num_signal_samples]
        """
        frame_len = self._model_settings['window_samples']
        frame_step = self._model_settings['window_stride_samples']
        fft_size = self._model_settings['fft_size']
        
        
        spect = tf.contrib.signal.stft(signals, frame_len, frame_step, fft_size)
        spect_mag = tf.abs(spect)
        mel_spect = tf.tensordot(spect_mag, self._mel_matrix, 1)
        
        if debug_stuff:
            return spect, mel_spect
        else:
            return mel_spect
        

    def get_data(self, debug_stuff = False):
        """
            Applies random transforms, and returns audio as spectrograms
        """
        signals, labels = self._signals, self._labels
        background = self._next_background[0:signals.shape[1]]
        
        #random volume, at most 'background volume', but non-zero only for 'background_percent' of samples
        foreground_rms = tf.sqrt(tf.reduce_mean(signals*signals, axis=1))
        background_volume =   tf.random_uniform((tf.shape(signals)[0],), 0, self._model_settings['background_volume'])\
                            * tf.cast(tf.random_uniform((tf.shape(signals)[0],), 0, 100)<self._model_settings['background_percent'], tf.float32)\
                            * (foreground_rms/FLAGS['target_rms'])
                
        #randomly shift signals in time by +/- max shift samples        
        max_shift = self._model_settings['max_shift_samples']
        clip_samples = model_settings['clip_samples']
        signals = tf.pad(signals, ((0,0), (max_shift, max_shift)), 'constant')
        offset = tf.random_uniform((1,), maxval=max_shift*2, dtype=tf.int32)[0]
        signals = signals[:, offset:offset+clip_samples]
                         
        
        #volume = 0 for silence
        foreground_volume = tf.cast(tf.not_equal(labels, self.label_to_id[self.silence_label]), tf.float32)
        foreground_volume = foreground_volume[:, None]
        
        signals_mixed = foreground_volume * signals + background_volume[:, None] @ background[None, :]
        signals_mixed = tf.clip_by_value(signals_mixed, -1, 1)

        if debug_stuff:
            spect, mel_spect = self.signal_to_mel(signals_mixed, debug_stuff)
            return signals, background, signals_mixed, spect, mel_spect, labels
        else:
            mel_spect = self.signal_to_mel(signals_mixed, debug_stuff)
            return mel_spect, labels
        
ap = AudioProcessor(FLAGS['dataset_dir'], FLAGS['background_dir'], FLAGS['wanted_words'], model_settings, training_settings)    

Preparing mel matrix
Preparing mel matrix finished
Indexing audio files
Examples per word:
{'zero': 2376, 'one': 2370, 'two': 2373, 'three': 2356, 'four': 2372, 'five': 2357, 'six': 2369, 'seven': 2377, 'eight': 2352, 'nine': 2364, '_silence_': 10413, '_unknown_': 2367}
Indexing audio files finished
Loading backgrounds
Loading backgrounds finished


In [122]:
def reconstruct_audio_from_mel(mel_spect, model_settings, true_spect = None):
    mel_inv = np.linalg.pinv(ap._mel_matrix.eval())
    spect_mag_reconstructed = (mel_spect @ mel_inv)
    
    if true_spect is not None:
        phase = tf.angle(tf.constant(true_spect))
    else:
        phase = tf.random_uniform(spect_mag_reconstructed.shape, 2 * np.pi)
        
    spect_reconstructed = tf.complex(spect_mag_reconstructed * tf.cos(phase), spect_mag_reconstructed * tf.sin(phase))
    audio = tf.contrib.signal.inverse_stft(spect_reconstructed, model_settings['window_samples'], 
                                           model_settings['window_stride_samples'], model_settings['fft_size'])
    return audio.eval()

In [123]:
signals, background, signals_mixed, spect, mel_spectras, labels = ap.get_data(True)

In [124]:
play_example = True
example_id = 0
if play_example:
    ap.iter.initializer.run(feed_dict={ap.x_src: ap.train_set[0], ap.y_src: ap.train_set[1]})
    vals = isess.run((signals, background, signals_mixed, spect, mel_spectras, labels))
    s, b, sm, sp, ms, lab = [v[example_id] for v in vals]

    print(ap.id_to_label[lab])
    print(np.max(np.abs(sm)))
    play_audio(FLAGS['sampling_rate'], reconstruct_audio_from_mel(ms, model_settings, sp) )#reconstructed with true phase
#     play_audio(FLAGS['sampling_rate'], reconstruct_audio_from_mel(ms, model_settings) )    #reconstructed without phase
#     play_audio(FLAGS['sampling_rate'], sm )                                                #mixed signals before stft    

eight
0.7332775


In [125]:
def conv_model(model_settings, fingerprint, num_labels, reuse=tf.AUTO_REUSE):
    if fingerprint.shape[1] != model_settings['lenght_in_samples'] or fingerprint.shape[2] != model_settings['mel_num_bins']:
        raise RuntimeError("Unexpected input: " + str(fingerprint.shape))

    h = tf.expand_dims(fingerprint, -1) #Add dimension at the end as as channels
    h = tf.layers.conv2d(h, 64, [20, 8], activation=tf.nn.relu, name='conv1', reuse=reuse) #[time span, freq_span]
    h = tf.layers.max_pooling2d(h, [1,3], [1,3])
    h = tf.layers.conv2d(h, 64, [10, 4], activation=tf.nn.relu, name='conv2', reuse=reuse)
    h = tf.layers.flatten(h)
    h = tf.layers.dense(h, 128, activation=tf.nn.relu, name='dense1', reuse=reuse)
    h = tf.layers.dense(h, num_labels, name='dense2', reuse=reuse)
    scores = h
    return scores
        
scores = conv_model(model_settings, mel_spectras, len(ap.id_to_label))
xent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=scores)
loss = tf.reduce_mean(xent)
loss_summary = tf.summary.scalar("loss", loss)
pred = tf.argmax(scores, 1, output_type=tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred,labels), tf.float32))
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
merged_summary = tf.summary.merge([loss_summary, accuracy_summary])

In [126]:
with tf.variable_scope("", reuse=tf.AUTO_REUSE):
    optim = tf.train.AdamOptimizer(FLAGS['lr'])
    step = optim.minimize(loss)
global_step = tf.train.get_or_create_global_step()
increment_step = tf.assign(global_step, global_step+1)

saver = tf.train.Saver(tf.global_variables())
train_writer = tf.summary.FileWriter('train_log', isess.graph)
valid_writer = tf.summary.FileWriter('valid_log')

In [127]:
tf.global_variables_initializer().run()

In [128]:
saver.restore(isess, "checkpoints/conv.ckpt-2020")

INFO:tensorflow:Restoring parameters from checkpoints/conv.ckpt-2020


## Training

In [130]:
epochs = 20
for epoch in range(epochs):
    #train
    ap.iter.initializer.run(feed_dict={ap.x_src: ap.train_set[0], ap.y_src: ap.train_set[1]})
    e_sum = 0
    mean_acc = 0
    for i in count():
        try:
            train_summary, loss_val, acc_val, _, _ = isess.run([merged_summary, loss, accuracy, step, increment_step])
            e_sum += loss_val
            mean_acc +=acc_val
            train_writer.add_summary(train_summary, global_step.eval())
            print("{}/{}: loss {:.3f} acc {:.3f}".format(i, 
                                                         len(ap.train_set[0])//FLAGS['batch_size'],
                                                         loss_val, 
                                                         acc_val),
                  end='\r')
        except tf.errors.OutOfRangeError:
            break
    print()
    print("Epoch {}:{}loss {:.3f} acc {:.3f}".format(epoch," "*30, (e_sum/i), (mean_acc/i)))
    print("Saving to {}-{}".format(FLAGS['checkpoint_dir'], global_step.eval()))
    saver.save(isess, FLAGS['checkpoint_dir'], global_step=global_step.eval())
    
    #valid
    ap.iter.initializer.run(feed_dict={ap.x_src: ap.valid_set[0], ap.y_src: ap.valid_set[1]})
    e_sum = 0
    mean_acc = 0
    for i in count():
        try:
            valid_summary, loss_val, acc_val, = isess.run([merged_summary, loss, accuracy])
            e_sum += loss_val
            mean_acc += acc_val
            valid_writer.add_summary(valid_summary, epoch)
        except tf.errors.OutOfRangeError:
            break
    print("Validation {}:{}loss {:.3f} acc {:.3f}".format(epoch," "*50, (e_sum/i), (mean_acc/i)))


256/256: loss 0.150 acc 0.941
Epoch 0:                              loss 0.299 acc 0.918
Saving to checkpoints/conv.ckpt-4847
Validation 0:                                                  loss 0.299 acc 0.921
256/256: loss 0.305 acc 0.912
Epoch 1:                              loss 0.277 acc 0.923
Saving to checkpoints/conv.ckpt-5104
Validation 1:                                                  loss 0.289 acc 0.923
256/256: loss 0.199 acc 0.941
Epoch 2:                              loss 0.281 acc 0.923
Saving to checkpoints/conv.ckpt-5361
Validation 2:                                                  loss 0.295 acc 0.924
256/256: loss 0.364 acc 0.882
Epoch 3:                              loss 0.250 acc 0.931
Saving to checkpoints/conv.ckpt-5618
Validation 3:                                                  loss 0.373 acc 0.910
256/256: loss 0.080 acc 0.971
Epoch 4:                              loss 0.235 acc 0.935
Saving to checkpoints/conv.ckpt-5875
Validation 4:                     

## Demo

In [131]:
def single_file_processing_graph(filename):
    audio_tensor = ap.load_wav(filename, model_settings['clip_samples'])
    return raw_data_processing_graph(audio_tensor)
    
def raw_data_processing_graph(audio_tensor):
    #audio_tensor [samples, ] float32 tensor
    audio_tensor = audio_tensor[None, :]
    mel_spect = ap.signal_to_mel(audio_tensor)
    nn_scores = conv_model(model_settings, mel_spect, len(ap.id_to_label))
    pred_label_id = tf.argmax(nn_scores, 1, output_type=tf.int32)
    return nn_scores, pred_label_id


filename_input = tf.placeholder(tf.string, shape=[])
audio_tensor_input = tf.placeholder(tf.float32, shape=[model_settings['clip_samples']])

In [ ]:
#Test some audio from files - by default from training set
nn_scores, pred_label_id = single_file_processing_graph(filename_input)
for i in range(10):
    path, true_label_id = ap.train_set[0][i], ap.train_set[1][i]
    if true_label_id == ap.label_to_id[ap.silence_label]:
        continue #here no tracks are mixed/no volume adjustment, so silence doesn't make sense
    pred_label_id_val = pred_label_id.eval(feed_dict={filename_input: path})[0]
    mark = "" if true_label_id == pred_label_id_val else "!"
    
    print("{}Predicted: {}, true {}".format(mark, ap.id_to_label[pred_label_id_val], ap.id_to_label[true_label_id]))

In [132]:
#Record some audio and test
nn_scores, pred_label = raw_data_processing_graph(audio_tensor_input)

In [140]:
fs = FLAGS['sampling_rate']
output = record_audio(FLAGS['clip_time_ms']/1000, fs, False)
play_audio(fs, output)
output_padded = np.pad(output, (0, model_settings['clip_samples'] - output.shape[0]), 'constant')
label_id = pred_label.eval(feed_dict={audio_tensor_input:output_padded})[0]
print("I heard {}".format(ap.id_to_label[label_id]))

recording...
finished recording, took 1.509677
I heard _silence_


In [161]:
from threading import Event, Thread
from math import ceil


event = Event()

buffer = None
frames = None
samples_ready = 0
alive = True
def record_audio2(seconds, rate, event):       
    global buffer, frames, samples_ready
    FORMAT = pyaudio.paFloat32
    CHANNELS = 1
    CHUNK = 1024
    
    frames = int(rate / CHUNK * seconds)
    buffer = np.zeros(CHUNK*frames)
    event.set()
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=rate, input=True,
                    frames_per_buffer=CHUNK)

    print("recording...")
    for i in range(frames):
        data = stream.read(CHUNK)
        buffer[i*CHUNK: (i+1)*CHUNK] = np.frombuffer(data, dtype=np.float32)
        samples_ready = (i+1)*CHUNK
        event.set()
        if not alive: break
    print("finished recording")

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

recorder = Thread(target=record_audio2, args=(10, FLAGS['sampling_rate'], event))
recorder.start()

event.wait()
event.clear()

run_every_ms = 200
run_every_samples =  int(ceil(FLAGS['sampling_rate']*run_every_ms/1000))
next_start_sample = 0
next_end_sample = model_settings['clip_samples']
last_id = -1
probability_dist = tf.nn.softmax(nn_scores)
try:
    while next_end_sample < buffer.size:
        event.wait()
        while samples_ready > next_end_sample:
            data = buffer[next_start_sample:next_end_sample]
            probs, label_id = isess.run((probability_dist, pred_label), feed_dict={audio_tensor_input:data})
            label_id = label_id[0]
            prob = probs[0][label_id]
            if prob>0.6 and label_id != ap.label_to_id[ap.silence_label] and label_id != last_id:
                print("I heard {} with p={:.2f}".format(ap.id_to_label[label_id], prob))
                last_id = label_id
            next_start_sample += run_every_samples
            next_end_sample   += run_every_samples
        event.clear()
except KeyboardInterrupt:
    alive = False
    raise
    

recorder.join()

recording...
finished recording


In [ ]:
ding = np.cos(2 * np.pi * 440/FLAGS['sampling_rate'] * np.arange(4000))*0.1

next_start_sample = 0
next_end_sample = model_settings['clip_samples']



while samples_ready > next_end_sample:
    data = buffer[next_start_sample:next_end_sample]
    play_audio(FLAGS['sampling_rate'], ding)
    play_audio(FLAGS['sampling_rate'], data)
    probs, label_id = isess.run((probability_dist, pred_label), feed_dict={audio_tensor_input:data})
    label_id = label_id[0]
    prob = probs[0][label_id]

    print("I heard {} with p={:.2f}".format(ap.id_to_label[label_id], prob))
    last_id = label_id
    sleep(0.5)
    next_start_sample += run_every_samples
    next_end_sample   += run_every_samples


### Random tests

In [ ]:
data = [ ['ą','as'], ['b', 'bas'] ]
p = tf.placeholder(tf.string, [None, 2])
pa = p+'a'
pa = sess.run(pa, feed_dict={p:data})
str(pa[0][0].decode("utf-8"))